In [ ]:
"""qsvd-v4.py"""

# A = Original matrix
# B = Square root of Matrix A
# U, V, L = Matrices obtained after SVD
# Ut, Vt, Lt = Truncated U, V, L matrices
# Bnp = Matrix obtained from U, V, L with no phase added or truncation done
# Bp = Matrix obtained from U, V, L after truncation and phase added
# Bt = Matrix obtained from U, V, L after truncation, no phase added

""" Import Libraries """
import numpy as np
import scipy.optimize as optimize
import time

### Method 1

In [ ]:
""" Optimization Algorithm """
""" New Matrix """
def newMat_1(x, Ut, Lt, Vt, k):
  U_new = np.zeros((Ut.shape), dtype=np.cfloat)
  if k==2:
    l0,l1 = Lt[0], Lt[1]
    for i in range(Ut.shape[0]):
      U_new[i][0] = np.cos(x[i]) / l0
      U_new[i][1] = (np.sin(x[i]) / l1) * np.exp(1j*x[i+Ut.shape[0]])
  elif k==3:
    l0,l1,l2 = Lt[0], Lt[1], Lt[2]
    for i in range(Ut.shape[0]):
      U_new[i][0] = np.cos(x[i]) / l0
      U_new[i][1] = (np.sin(x[i]) / l1) * (np.cos(x[i+Ut.shape[0]])) * np.exp(1j*x[i+2*Ut.shape[0]])
      U_new[i][2] = (np.sin(x[i]) / l2) * (np.sin(x[i+Ut.shape[0]])) * np.exp(1j*x[i+3*Ut.shape[0]])
  else:
    l0,l1,l2,l3 = Lt[0], Lt[1], Lt[2], Lt[3]
    for i in range(Ut.shape[0]):
      U_new[i][0] = (np.cos(x[i]) / l0) * (np.cos(x[i+Ut.shape[0]]))
      U_new[i][1] = (np.cos(x[i]) / l1) * (np.sin(x[i+Ut.shape[0]])) * np.exp(1j*x[i+3*Ut.shape[0]])
      U_new[i][2] = (np.sin(x[i]) / l2) * (np.cos(x[i+2*Ut.shape[0]])) * np.exp(1j*x[i+4*Ut.shape[0]])
      U_new[i][3] = (np.sin(x[i]) / l3) * (np.sin(x[i+2*Ut.shape[0]])) * np.exp(1j*x[i+5*Ut.shape[0]])
  return U_new


""" Cost Function """
def costFn(x, Ut, Lt, Vt, B, k):
    U_new = newMat_1(x, Ut, Lt, Vt, k)
    Bp = np.dot(np.dot(U_new,np.diag(Lt)), Vt)
    loss = np.linalg.norm(B**2 - np.abs(Bp)**2)
    return (loss)


""" Calculate Results """
def calcResults(k):
  print("k = ", str(k))
  n = 4
  for m in range(4, 12):
    print("m = ",m,", n = ",n)
    res = np.zeros((100,3))
    for i in range(100):
        A = np.random.rand(m, n)
        for j in range(m): A[j] /= sum(A[j])
        B = np.sqrt(A)
        U, L, V = np.linalg.svd(B, full_matrices=False)
        initial_guess = np.ones((2*m*(k-1),), dtype=np.longdouble)
        Ut = U[:, :k]
        Vt = V[:k]
        Lt = L[:k]
        Bt = np.dot(np.dot(Ut,np.diag(Lt)), Vt)
        start1 = time.time()
        result = optimize.minimize(fun=costFn, x0=initial_guess, args=(Ut,Lt,Vt,B,k),
                                    tol=1e-7, method='Nelder-Mead', options={'maxiter':1e+10})
        end1 = time.time()
        res[i][0] = (np.linalg.norm(B**2 - Bt**2))
        res[i][1] = costFn(result.x,Ut,Lt,Vt,B,k)
        res[i][2] = end1-start1
        if(i%10==0):
          print(i, " ", end='')
    print('\n')
    filename= ('data/k{0}_test/m{1}_n{2}_1.npy'.format(str(k),str(m),str(n)))
    np.save(filename, res)
  return

In [ ]:
calcResults(k=2)

In [ ]:
calcResults(k=3)

### Method 2

In [ ]:
""" Optimization Algorithm """
""" New Matrix """
def newMat_2(x, Ut, Lt, Vt, k):
  U_new = np.zeros((Ut.shape), dtype=np.cfloat)
  if k==2:
    l0,l1 = Lt[0], Lt[1]
    U_new[0] = np.cos(x[0]) / l0
    U_new[1] = (np.sin(x[0]) / l1) * np.exp(1j*x[1])
  elif k==3:
    l0,l1,l2 = Lt[0], Lt[1], Lt[2]
    U_new[0] = np.cos(x[0]) / l0
    U_new[1] = (np.sin(x[0]) / l1) * (np.cos(x[1])) * np.exp(1j*x[2])
    U_new[2] = (np.sin(x[0]) / l2) * (np.sin(x[1])) * np.exp(1j*x[3])
  else:
    l0,l1,l2,l3 = Lt[0], Lt[1], Lt[2], Lt[3]
    U_new[0] = (np.cos(x[0]) / l0) * (np.cos(x[1]))
    U_new[1] = (np.cos(x[0]) / l1) * (np.sin(x[1])) * np.exp(1j*x[3])
    U_new[2] = (np.sin(x[0]) / l2) * (np.cos(x[2])) * np.exp(1j*x[4])
    U_new[3] = (np.sin(x[0]) / l3) * (np.sin(x[2])) * np.exp(1j*x[5])
  return U_new


""" Cost Function """
def costFn(x, Ut, Lt, Vt, B, k):
    U_new = newMat_2(x, Ut, Lt, Vt, k)
    Bp = np.dot(np.dot(U_new,np.diag(Lt)), Vt) 
    loss = np.linalg.norm(B**2 - np.abs(Bp)**2)
    return (loss)

""" Calculate Results """
def calcResults(k):
  print("k = ", str(k))
  n = 4
  for m in range(4, 12):
    print("m = ",m,", n = ",n)
    res = np.zeros((100, 3))
    for i in range(100):
      A = np.random.rand(m, n)
      for j in range(m): A[j] /= sum(A[j])
      B = np.sqrt(A)
      U, L, V = np.linalg.svd(B, full_matrices=False)
      initial_guess = np.ones((2*(k-1),), dtype=np.longdouble)
      Ut = U[:, :k]
      Vt = V[:k]
      Lt = L[:k]
      Bt = np.dot(np.dot(Ut,np.diag(Lt)), Vt)
      U_new = np.zeros(Ut.shape, dtype=np.cfloat)
      start1 = time.time()
      for row in range(Ut.shape[0]):
        result = optimize.minimize(fun=costFn, x0=initial_guess, args=(Ut[row, :],Lt,Vt,B[row,:],k),
                                  tol=1e-7, method='Nelder-Mead', options={'maxiter':1e+10})
        U_new[row,:] = newMat_2(result.x, Ut[row, :],Lt,Vt,k)
      end1 = time.time()
      Bp = np.dot(np.dot(U_new,np.diag(Lt)), Vt) 
      
      res[i][0] = (np.linalg.norm(B**2 - Bt**2))
      res[i][1] = (np.linalg.norm(B**2 - ((Bp)*np.conj(Bp))))
      res[i][2] = end1 - start1
      if i%10==0: print(i, end=' ')
    print('\n')
    filename= ('data/k{0}_test/m{1}_n{2}_2.npy'.format(str(k),str(m),str(n)))
    np.save(filename, res)
  return

In [ ]:
calcResults(k=2)

In [ ]:
calcResults(k=3)